In [13]:
import pandas as pd, numpy as np, sys, os

# HistSquareDiff

In [2]:
def HistSquareDiff(exp, ctrl, factor=1):
    """The actual workhorse HistDiff scoring function"""

    # we transpose twice to ensure the final result is a 1 x m feature score vector
    ctrl_meanProxy = (np.arange(1, ctrl.shape[0] + 1) * ctrl.T).T.sum(axis=0)
    exp_meanProxy = (np.arange(1, exp.shape[0] + 1) * exp.T).T.sum(axis=0)

    # evaluate where and when to adjust the score to be negative
    negScore = np.where(ctrl_meanProxy > exp_meanProxy, -1, 1)
    diff = ctrl - (exp.T * factor)
    diff **= 2

    return diff.sum(axis=1) * negScore


In [3]:
exp_mat = np.array([[1, 2],
                    [2, 3],
                    [3, 4]])
crt_mat = np.array([2,3,4])

HistSquareDiff(exp=exp_mat, ctrl=crt_mat, factor=0.5)

array([-12.5 ,   7.25])

# getMinMaxPlate

In [10]:
def getMinMaxPlate(
    chunks: pd.io.parsers.readers.TextFileReader , id_col: list[str] | str, verbose=True, probOut=None
) -> tuple[pd.DataFrame, list[str], pd.DataFrame]:
    """Gets the min and max of the features and returns features that are useful"""
    xlow = []
    xHigh = []

    feats: list[str] = []

    for count, chunk in enumerate(chunks, start=1):
        currDf = chunk
        currDf.set_index(id_col, inplace=True)
        currDf = currDf.replace(to_replace=-np.inf, value=np.nan)
        currDf = currDf.replace(to_replace=np.inf, value=np.nan)

        if count == 1:
            feats = currDf.columns.to_list()

        xlow.append(currDf.min(axis=0).to_list())
        xHigh.append(currDf.max(axis=0).to_list())

    xlow = pd.DataFrame(xlow).min(axis=0)
    xhigh = pd.DataFrame(xHigh).max(axis=0)

    # adjusting the high ranges
    xhigh[xhigh == xlow] = xlow[xhigh == xlow] + xlow[xhigh == xlow] * 0.5
    xhigh[xhigh == xlow] = xlow[xhigh == xlow] + 1

    min_max = pd.DataFrame(
        {"xlow": xlow.to_list(), "xhigh": xhigh.to_list()}, index=feats
    )

    bad_features = {
        feature: "noValues"
        for feature in min_max[
            min_max.apply(lambda x: all(np.isnan(x)), axis=1)
        ].index.values.tolist()
    }
    problematic_features_df = None
    if bad_features and verbose:
        print(
            f"MinMax: No values have been found in the following features: "
            f'{" | ".join(bad_features.keys())}',
            file=sys.stderr,
        )
        problematic_features_df = pd.Series(bad_features, name="histdiff_issue")
        if probOut is not None:
            problematic_features_df.to_csv(f"{probOut}_problematicFeats.csv")

    # Get good features and min_max table
    min_max = min_max[~min_max.apply(lambda x: all(np.isnan(x)), axis=1)]
    good_features = min_max.index.values.tolist()
    print(f"length of good features is: {len(good_features)}", file=sys.stderr)

    return (min_max, good_features, problematic_features_df)


In [14]:
file = "/home/derfelt/LokeyLabFiles/TargetMol/GR_followup/dataset/cell_by_cell_data/024ebc52-9579-11ef-b032-02420a00010f_cellbycell_HD_input.tsv.gz"
file = pd.read_table(file, compression='infer', chunksize=10000)
id_col = 'id'

test_out, _, _ = getMinMaxPlate(chunks=file, id_col=id_col)

display(test_out)

MinMax: No values have been found in the following features: Nuclei-Membrane_Region_HOECHST_33342_(global)_Profile_3/5 | Nuclei-Membrane_Region_HOECHST_33342_(global)_Profile_3/5_SP-Filter | Nuclei-Membrane_Region_HOECHST_33342_(global)_Profile_3/5_SER-Bright | Nuclei-Membrane_Region_HOECHST_33342_(global)_Profile_3/5_SER-Dark | Nuclei-Membrane_Region_HOECHST_33342_(global)_Profile_3/5_SER-Ridge | Nuclei-Membrane_Region_HOECHST_33342_(global)_Profile_3/5_SER-Valley | Nuclei-Membrane_Region_HOECHST_33342_(global)_Profile_3/5_SER-Spot | Nuclei-Membrane_Region_HOECHST_33342_(global)_Profile_3/5_SER-Hole | Nuclei-Membrane_Region_HOECHST_33342_(global)_Profile_3/5_SER-Saddle | Nuclei-Membrane_Region_HOECHST_33342_(global)_Profile_3/5_SER-Edge | Nuclei-Membrane_Region_TRITC_(global)_Profile_3/5 | Nuclei-Membrane_Region_TRITC_(global)_Profile_3/5_SP-Filter | Nuclei-Membrane_Region_TRITC_(global)_Profile_3/5_SER-Bright | Nuclei-Membrane_Region_TRITC_(global)_Profile_3/5_SER-Dark | Nuclei-Membr

,xlow,xhigh
Nuclei-Intensity_Nucleus_Region_HOECHST_33342_(global)_Mean,1318.740000,53703.800000
Nuclei-Intensity_Nucleus_Region_HOECHST_33342_(global)_StdDev,119.789000,14356.600000
Nuclei-Intensity_Nucleus_Region_HOECHST_33342_(global)_Median,1201.000000,56188.000000
Nuclei-Intensity_Nucleus_Region_HOECHST_33342_(global)_Maximum,1784.000000,65535.000000
Nuclei-Intensity_Nucleus_Region_HOECHST_33342_(global)_Minimum,231.000000,25728.000000
...,...,...
Nuclei-Membrane_Region_Alexa_647_(global)_Haralick_Contrast_2_px,0.000000,37.722600
Nuclei-Membrane_Region_Alexa_647_(global)_Haralick_Sum_Variance_2_px,0.000000,52.632100
Nuclei-Membrane_Region_Alexa_647_(global)_Haralick_Homogeneity_2_px,0.018185,1.000000
Nuclei-Membrane_Region_Alexa_647_(global)_Gabor_Min_2_px_w2,0.000027,0.005261
